<a href="https://colab.research.google.com/github/dhirajsuvarna/pointnet.pytorch/blob/master/notebooks/PointNet_Training_DMU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement PointNet on DMU-Dataset



### Setup the google drive with dataset and code

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mydrive


In [8]:
cd /content/mydrive/My\ Drive/Projects/pointnet

/content/mydrive/My Drive/Projects/pointnet


In [9]:
cd pointnet.pytorch/

/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


In [13]:
#for the first time
#!git clone https://github.com/dhirajsuvarna/pointnet.pytorch.git
! git pull

Updating bb3d623..41ee72b
Fast-forward
 .../pcd_utils/create_train_test_shuffle.py         | 87 ++++++++++++++++++++++
 dataset_utils/pcd_utils/create_uniform_pcd.py      | 40 ++++++++++
 dataset_utils/readme.md                            | 15 ++++
 pointnet/dataset.py                                |  5 +-
 train_classification.py                            | 41 +++++++---
 5 files changed, 175 insertions(+), 13 deletions(-)
 create mode 100644 dataset_utils/pcd_utils/create_train_test_shuffle.py
 create mode 100644 dataset_utils/pcd_utils/create_uniform_pcd.py
 create mode 100644 dataset_utils/readme.md


In [0]:
cd /content/mydrive/My\ Drive/Projects/pointnet/dataset

/content/mydrive/My Drive/Projects/pointnet/dataset


In [0]:
!unzip dataset_PCD_5000.zip

Archive:  dataset_PCD_5000.zip
   creating: dataset_PCD_5000/
   creating: dataset_PCD_5000/Actuator/
  inflating: dataset_PCD_5000/Actuator/Actuator_0.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_1.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_10.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_11.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_12.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_13.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_14.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_15.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_16.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_17.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_18.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_19.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_2.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_20.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuator_21.pcd  
  inflating: dataset_PCD_5000/Actuator/Actuato

### try to get max gpu

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=8dd00f879db4783d97f015fce06cedbac1416865705d6f01127a98baa73f7f4d
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 158.5 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


# Install the Python Pacakages

In [2]:
cd /content/mydrive/My\ Drive/Projects/pointnet/pointnet.pytorch


/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


In [3]:
!pip install plyfile

In [4]:
!python -m pip install git+https://github.com/DanielPollithy/pypcd.git

  Cloning https://github.com/DanielPollithy/pypcd.git to /tmp/pip-req-build-rk4fmwdm
  Running command git clone -q https://github.com/DanielPollithy/pypcd.git /tmp/pip-req-build-rk4fmwdm
  Created wheel for pypcd: filename=pypcd-0.1.3-py2.py3-none-any.whl size=23397 sha256=b2b619644f002cd23671dd98ca4434fb63c96b77fa8d7b62e980c99b4c7e6c19
  Stored in directory: /tmp/pip-ephem-wheel-cache-uz101ws9/wheels/1a/a3/7a/62170df4ad89eaaf95e7c9d8ee64808580cde1b9f8c22d8562
  Created wheel for python-lzf: filename=python_lzf-0.2.4-cp36-cp36m-linux_x86_64.whl size=22080 sha256=c7eb89c10b7e187a6d4363c9619cec0b3229d3a891be8ce3bf5cc75eff3eb756
  Stored in directory: /root/.cache/pip/wheels/67/92/2e/de0656a80f99649ca610c10d96aef9aed3ef19697606bcb94f
Successfully built pypcd python-lzf


### Create Train Test Split

In [0]:
!python create_train_test_shuffle.py

debug: Actuator => totalFiles: 38, train: 30 + test: 8 = 38 
debug: AirFilter => totalFiles: 32, train: 25 + test: 7 = 32 
debug: Battery => totalFiles: 46, train: 36 + test: 10 = 46 
debug: Bearing => totalFiles: 128, train: 102 + test: 26 = 128 
debug: Brake => totalFiles: 39, train: 31 + test: 8 = 39 
debug: Camshaft => totalFiles: 46, train: 36 + test: 10 = 46 
debug: Clutch => totalFiles: 61, train: 48 + test: 13 = 61 
debug: Cooler => totalFiles: 30, train: 24 + test: 6 = 30 
debug: Coupling => totalFiles: 43, train: 34 + test: 9 = 43 
debug: Crankshaft => totalFiles: 81, train: 64 + test: 17 = 81 
debug: CuttingTool => totalFiles: 41, train: 32 + test: 9 = 41 
debug: ElectricMotor => totalFiles: 142, train: 113 + test: 29 = 142 
debug: ElectronicBoard => totalFiles: 37, train: 29 + test: 8 = 37 
debug: ExhaustPipe => totalFiles: 20, train: 16 + test: 4 = 20 
debug: Gear => totalFiles: 18, train: 14 + test: 4 = 18 
debug: GearWheel => totalFiles: 91, train: 72 + test: 19 = 91 
de

In [0]:
!cat dataset_PCD_5000/train_test_split.json | head 

{
    "train": {
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_15.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_27.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_21.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_12.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_35.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_13.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_32.pcd": "Actuator",
        "/content/mydrive/My Drive/Projects/pointnet/dataset/dataset_PCD_5000/Actuator/Actuator_17.pcd": "Actuator",


In [0]:
cd ../pointnet.pytorch/

/content/mydrive/My Drive/Projects/pointnet/pointnet.pytorch


# Start Training 

In [17]:
!python train_classification.py --nepoch 50 \
                                --batchSize 32 \
                                --num_points 4000 \
                                --dataset_type dmunet \
                                --dataset /content/mydrive/My\ Drive/Projects/pointnet/dataset/dataset_PCD_5000 

Streaming output truncated to the last 5000 lines.
       device='cuda:0')
NLL-Loss 0.2074897736310959
Processing Files: ('GearWheel_85.pcd', 'Screw_185.pcd', 'Pulley_32.pcd', 'Piston_23.pcd', 'RodPistonAssembly_32.pcd', 'Bearing_89.pcd', 'WingNut_4.pcd', 'Pulley_61.pcd', 'Screw_208.pcd', 'Brake_11.pcd', 'Nut_33.pcd', 'Washer_15.pcd', 'Brake_36.pcd', 'WingNut_25.pcd', 'Nut_38.pcd', 'ElectricMotor_69.pcd', 'Helix_16.pcd', 'Coupling_33.pcd', 'Bearing_93.pcd', 'Brake_2.pcd', 'SparkPlug_19.pcd', 'Camshaft_19.pcd', 'Cooler_2.pcd', 'Switch_41.pcd', 'SparkPlug_29.pcd', 'ElectricMotor_108.pcd', 'Screw_179.pcd', 'ElectricMotor_22.pcd', 'Piston_0.pcd', 'Bearing_35.pcd', 'ElectricMotor_116.pcd', 'Screw_49.pcd')
[39: 15/47] train loss: 0.216115 accuracy: 0.906250
Predictions: tensor([16, 24, 20, 19, 23,  3, 31, 16, 24,  4, 18, 30,  4, 31, 18, 12, 17,  9,
         3,  4, 26, 25,  8, 27, 26, 12, 24, 12, 19, 16, 12, 24],
       device='cuda:0')
Targets: tensor([16, 24, 20, 19, 23,  3, 31, 20, 24,  4,

### Dataset Analysis

Anaysis of number of points present in each of the model